In [2]:
import numpy as np
from abc import ABC, abstractmethod

In [3]:
class Solver(ABC):
    """A solver. It may be initialized with some hyperparameters."""

    @abstractmethod
    def get_parameters(self):
        """Returns a dictionary of hyperparameters"""
        ...

    @abstractmethod
    def solve(self, problem, x0, *args, **kwargs):
        """
        A method that solves the given problem for given initial solution.
        It may accept or require additional parameters.
        Returns the solution and may return additional info.
        """



In [19]:
class GradientDescent(Solver):
    def __init__(self, beta, iterations):
        self.beta = beta
        self.iterations = iterations

    def get_parameters(self):
        dict = {}
        dict["Beta"] = self.beta
        dict["Iterations"] = self.iterations
        return dict
    
    def solve(self, problem, x0):
        while self.iterations > 0:
           self.iterations -= 1
           d = problem(x0)
           x0 = np.subtract(x0, d * self.beta)
           print(x0, d)
        return x0
    
    def gradient_f1(self, x):
        return x**3
    
    def gradient_f2(self, x0):
        x1, x2 = x0
        return np.array(2 * x1 * np.exp(-x1**2 - x2**2) + (x1-1) * np.exp(-(x1-1)**2 - (x2-1)**2),
                        2 * x2 * np.exp(-x1**2 - x2**2) + (x2-1) * np.exp(-(x1-1)**2 - (x2-1)**2))

In [20]:
a = GradientDescent(0.07, 10000)
a.solve(a.gradient_f2, (2.15, 3.19))

TypeError: GradientDescent.solve() takes 3 positional arguments but 5 were given